In [ ]:
import numpy as np
from PIL import Image, ImageEnhance
from PIL.ImageFilter import (
   BLUR, CONTOUR, DETAIL, EDGE_ENHANCE, EDGE_ENHANCE_MORE,
   EMBOSS, FIND_EDGES, SMOOTH, SMOOTH_MORE, SHARPEN
)
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
from torchvision import models
from torchvision import transforms as tf
import torch.nn.functional as F
import os

ROOT_PATH = os.path.dirname(os.path.abspath("__file__"))

IMAGE_FOLDER = 'content_images'

STYLE = 'style_images'

OUT_FOLDER = 'test'

# will download the pre-trained pytorch model for style transfer
vgg = models.vgg19(pretrained=True).features

for param in vgg.parameters():
    param.requires_grad_(False)

# set device to gpu if available. (CUDA required for fast processing)
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

# assigns model to device
vgg.to(device)

# sets params for image normalization
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# function for transforming images to arrays on tensor. the larger the value in tf.Resize, the longer it takes
def transformation(img):

    tasks = tf.Compose([tf.Resize(500),
                        tf.ToTensor(),
                        tf.Normalize(mean, std)])

    img = tasks(img)
    img = img.unsqueeze(0)

    return img

# paths to content and style images
content_img = Image.open(os.path.join(ROOT_PATH, IMAGE_FOLDER) + "\\cacti_1.jpg").convert('RGB')

style_img = Image.open(os.path.join(ROOT_PATH, STYLE) + "\\detailed_elephant.png").convert('RGB')

# optional image modifications see above PIL.ImageFilter imports for available options
# content_img = content_img.filter(FIND_EDGES)
#
# content_img = content_img.filter(EMBOSS)

# enhancer = ImageEnhance.Brightness(content_img)
#
# factor = 1.5
# content_img = enhancer.enhance(factor)

# assigns images to device
content_img = transformation(content_img).to(device)

style_img = transformation(style_img).to(device)


# function for converting tensor arrays back to images
def tensor_to_image(tensor):

    image = tensor.clone().detach()
    image = image.cpu().numpy().squeeze()

    image = image.transpose(1, 2, 0)

    image *= np.array(std) + np.array(mean)
    image = image.clip(0, 1)

    return image

# sets layers of network
LAYERS_OF_INTEREST = {'0': 'conv1_1',
                      '5': 'conv2_1',
                      '10': 'conv3_1',
                      '19': 'conv4_1',
                      '21': 'conv4_2',
                      '28': 'conv5_1'}


# function for applying model and extracting features for content and style images
def apply_model_and_extract_features(image, model):
    x = image

    features = {}

    for name, layer in model._modules.items():
        x = layer(x)

        if name in LAYERS_OF_INTEREST:
            features[LAYERS_OF_INTEREST[name]] = x

    return features

# sets extracted feature values
content_img_features = apply_model_and_extract_features(content_img, vgg)
style_img_features   = apply_model_and_extract_features(style_img, vgg)


# function for calculating gram matrix
def calculate_gram_matrix(tensor):

    _, channels, height, width = tensor.size()

    tensor = tensor.view(channels, height * width)

    gram_matrix = torch.mm(tensor, tensor.t())

    gram_matrix = gram_matrix.div(channels * height * width)

    return gram_matrix


style_features_gram_matrix = {layer: calculate_gram_matrix(style_img_features[layer]) for layer in style_img_features}

# sets weights for each layer of interest
weights = {'conv1_1': 1.0, 'conv2_1': 0.75, 'conv3_1': 0.35,
           'conv4_1': 0.25, 'conv5_1': 0.15}

target = content_img.clone().requires_grad_(True).to(device)

# sets optimization and relu values
optimizer = optim.Adam([target], lr=0.003)

# applies network model in whatever range specified for number of iterations
for i in range(1, 2000):

    target_features = apply_model_and_extract_features(target, vgg)

    content_loss = F.mse_loss (target_features['conv4_2'], content_img_features['conv4_2'])

    style_loss = 0
    for layer in weights:

        target_feature = target_features[layer]

        target_gram_matrix = calculate_gram_matrix(target_feature)
        style_gram_matrix = style_features_gram_matrix[layer]

        layer_loss = F.mse_loss (target_gram_matrix, style_gram_matrix)
        layer_loss *= weights[layer]

        _, channels, height, width = target_feature.shape

        style_loss += layer_loss

    total_loss = 1000000 * style_loss + content_loss

    if i % 500 == 0:
        print ('Epoch {}:, Style Loss : {:4f}, Content Loss : {:4f}'.format( i, style_loss, content_loss))

    optimizer.zero_grad()

    total_loss.backward()

    optimizer.step()

# displays original image and processed stylized image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(tensor_to_image(content_img))
ax2.imshow(tensor_to_image(target))

# converts target image from tensor array back to image
out = tensor_to_image(target)

# converts output image to writeable format, resizes to original value
img_out = Image.fromarray((out * 255).astype('uint8'), 'RGB')

# displays final image to be saved
img_out

In [ ]:
# use below to save test results

# the below will create an output directory and save the image as png

if not os.path.exists(os.path.join(ROOT_PATH, OUT_FOLDER)):
    os.mkdir(os.path.join(ROOT_PATH, OUT_FOLDER))

img_out.save(os.path.join(ROOT_PATH, OUT_FOLDER) + "\image_test" + ".png",
                 "PNG",
                 progressive=True,
                 quality=100,
                 optimize=True)